In [10]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [11]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hobyo,SO,2020-10-30 16:20:31,5.35,48.53,81.88,74,4,11.70
1,1,Ushuaia,AR,2020-10-30 16:19:04,-54.80,-68.30,48.20,76,75,29.97
2,2,Iralaya,HN,2020-10-30 16:20:31,15.00,-83.23,84.94,71,100,7.34
3,3,Taoudenni,ML,2020-10-30 16:20:28,22.68,-3.98,91.74,11,100,11.10
4,4,Luderitz,NaN,2020-10-30 16:20:31,-26.65,15.16,68.02,43,59,33.42


In [12]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [13]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [14]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [15]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 105


In [18]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna() 

In [19]:
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hobyo,SO,2020-10-30 16:20:31,5.35,48.53,81.88,74,4,11.70
2,2,Iralaya,HN,2020-10-30 16:20:31,15.00,-83.23,84.94,71,100,7.34
3,3,Taoudenni,ML,2020-10-30 16:20:28,22.68,-3.98,91.74,11,100,11.10
5,5,Hilo,US,2020-10-30 16:20:31,19.73,-155.09,75.20,88,90,3.36
6,6,Hot Springs,US,2020-10-30 16:20:31,34.50,-93.06,53.60,54,1,6.93
7,7,Kingaroy,AU,2020-10-30 16:20:31,-26.53,151.83,66.20,88,0,6.93
8,8,Bredasdorp,ZA,2020-10-30 16:20:31,-34.53,20.04,62.60,45,0,10.29
10,10,Sinnamary,GF,2020-10-30 16:20:12,5.38,-52.95,87.60,63,80,13.49
12,12,Los Llanos De Aridane,ES,2020-10-30 16:20:31,28.66,-17.92,75.20,73,40,13.87
16,16,Chatham,GB,2020-10-30 16:20:32,51.38,0.53,60.01,91,97,4.00


In [20]:
preferred_cities_df.count()

City_ID       392
City          392
Country       392
Date          392
Lat           392
Lng           392
Max Temp      392
Humidity      392
Cloudiness    392
Wind Speed    392
dtype: int64

In [21]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hobyo,SO,81.88,5.35,48.53,
2,Iralaya,HN,84.94,15.00,-83.23,
3,Taoudenni,ML,91.74,22.68,-3.98,
5,Hilo,US,75.20,19.73,-155.09,
6,Hot Springs,US,53.60,34.50,-93.06,
7,Kingaroy,AU,66.20,-26.53,151.83,
8,Bredasdorp,ZA,62.60,-34.53,20.04,
10,Sinnamary,GF,87.60,5.38,-52.95,
12,Los Llanos De Aridane,ES,75.20,28.66,-17.92,
16,Chatham,GB,60.01,51.38,0.53,


In [22]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}
   

In [23]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
hotel_df.count()

City          392
Country       392
Max Temp      392
Lat           392
Lng           392
Hotel Name    392
dtype: int64

In [28]:
hotel_df[(hotel_df["Hotel Name"]!="")].count()

City          360
Country       360
Max Temp      360
Lat           360
Lng           360
Hotel Name    360
dtype: int64

In [39]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [40]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [42]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [43]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))